### ConvolutionCyclopeptideSequencing(M, N, Spectrum)

In [ ]:
import numpy as np
from collections import Counter

def Expand(Peptides,AminoAcidMass):
    expanded = []
    for peptide in Peptides:
        for aa in AminoAcidMass.keys():
            expanded.append(peptide + aa)
    return expanded

def ParentMass(Spectrum):
    return max(Spectrum) 

def Mass(Peptide,AminoAcidMass):
    return sum(AminoAcidMass[aa] for aa in Peptide)

def CyclicSpectrum(Peptide, Alphabet, AminoAcidMass):
    PrefixMass = [0] * (len(Peptide) + 1)

    for i in range(1, len(Peptide) + 1):
        for s in Alphabet:
            if s == Peptide[i - 1]:
                PrefixMass[i] = PrefixMass[i - 1] + AminoAcidMass[s]

    peptideMass = PrefixMass[len(Peptide)]
    CyclicSpectrum = [0]

    for i in range(len(Peptide)):
        for j in range(i + 1, len(Peptide) + 1):
            CyclicSpectrum.append(PrefixMass[j] - PrefixMass[i])
            if i > 0 and j < len(Peptide):
                CyclicSpectrum.append(
                    peptideMass - (PrefixMass[j] - PrefixMass[i]))

    return sorted(CyclicSpectrum)

def CyclicScore(Peptide, Spectrum, Alphabet, AminoAcidMass):
    Spectrum_Teor = CyclicSpectrum(Peptide, Alphabet, AminoAcidMass)
    Score_final_Cyclic = 0
    p1 = 0
    p2 = 0
    
    while p1 < len(Spectrum_Teor) and p2 < len(Spectrum):
        if Spectrum_Teor[p1] == Spectrum[p2]:
            Score_final_Cyclic += 1
            p1 += 1
            p2 += 1
        elif Spectrum_Teor[p1] < Spectrum[p2]:
            p1 += 1
        elif Spectrum_Teor[p1] > Spectrum[p2]:
            p2 +=1
    return Score_final_Cyclic

def LinearSpectrum(Peptide, Alphabet, AminoAcidMass):
    PrefixMass = [0] * (len(Peptide) + 1)
    for i in range(1, len(Peptide) + 1):
        for s in Alphabet:
            if s == Peptide[i - 1]: 
                PrefixMass[i] = PrefixMass[i - 1] + AminoAcidMass[s]
    LinearSpectrum = [0]

    for i in range(len(Peptide)):
        for j in range(i + 1, len(Peptide) + 1):
            LinearSpectrum.append(PrefixMass[j] - PrefixMass[i])

    return sorted(LinearSpectrum)

def LinearScore(Peptide, Spectrum, Alphabet,AminoAcidMass):
    Spectrum_Teor = LinearSpectrum(Peptide, Alphabet, AminoAcidMass)
    LinearScore_final_linear = 0
    p1 = 0
    p2 = 0
    
    while p1 < len(Spectrum_Teor) and p2 < len(Spectrum):
        if Spectrum_Teor[p1] == Spectrum[p2]:
            LinearScore_final_linear += 1
            p1 += 1
            p2 += 1
        elif Spectrum_Teor[p1] < Spectrum[p2]:
            p1 += 1
        elif Spectrum_Teor[p1] > Spectrum[p2]:
            p2 +=1
    return LinearScore_final_linear

def Trim(Leaderboard, Spectrum, N, Alphabet, AminoAcidMass):
    LinearScores = []
    for j in range(1, len(Leaderboard) + 1):
        Peptide = Leaderboard[j - 1]
        LinearScores.append(LinearScore(Peptide, Spectrum, Alphabet, AminoAcidMass))

    sorted_indices = sorted(range(len(LinearScores)), key=lambda x: -LinearScores[x])
    Leaderboard = [Leaderboard[i] for i in sorted_indices]
    LinearScores = [LinearScores[i] for i in sorted_indices]

    for j in range(N + 1, len(Leaderboard) + 1):
        if LinearScores[j - 1] < LinearScores[N - 1]: 
            return Leaderboard[:j - 1]

    return Leaderboard

def LeaderboardCyclopeptideSequencing(Spectrum, N, Alphabet, AminoAcidMass):
    Leaderboard = {""}  # Conjunto contendo apenas o peptídeo vazio
    LeaderPeptides = []  # Lista para armazenar os peptídeos líderes de máximo score
    MaxScore = 0  # Maior score encontrado // MODIFICAÇÃO PARA ESSE EXERCÍCIO
    
    while Leaderboard:
        Leaderboard = Expand(Leaderboard, AminoAcidMass)  # Expande os peptídeos no Leaderboard
        for Peptide in list(Leaderboard):  # Itera sobre os peptídeos
            if Mass(Peptide,AminoAcidMass) == ParentMass(Spectrum):
                score = CyclicScore(Peptide, Spectrum, Alphabet, AminoAcidMass)
                if score > MaxScore:
                    MaxScore = score
                    LeaderPeptides = [Peptide]  # Atualiza os peptídeos líderes
                elif score == MaxScore:
                    LeaderPeptides.append(Peptide)
            elif Mass(Peptide,AminoAcidMass) > ParentMass(Spectrum):
                Leaderboard.remove(Peptide)  # Remove peptídeos com massa maior
        Leaderboard = Trim(Leaderboard, Spectrum, N, Alphabet, AminoAcidMass)

    return sorted(LeaderPeptides)  # Garantir saída ordenada

def format_peptide(peptide,AminoAcidMass):
    masses = [str(AminoAcidMass[aa]) for aa in peptide]
    return "-".join(masses)

def Spectral_Convolution(Spectrum):
    matriz_spec = np.zeros((len(Spectrum), len(Spectrum) - 1))

    for i in range(1, len(Spectrum)):
        for j in range(len(Spectrum) - 1):
            if Spectrum[i] > Spectrum[j]:
                matriz_spec[i][j] = Spectrum[i] - Spectrum[j]
    return matriz_spec

def format_ans(Spectrum):
    matriz_spec = Spectral_Convolution(Spectrum)
    result = []

    for linha in matriz_spec:
        for item in linha:
            if item != 0:
                result.append(int(item))
    return sorted(result)

def get_M(M,dic_result_sorted):
  if len(dic_result_sorted) <= M:
    return dic_result_sorted

  m_keys = list(dic_result_sorted.keys())[:M]
  m_values = list(dic_result_sorted.values())[:M]
  dic_final = {}

  for i in range(M):
    dic_final[m_keys[i]] = m_values[i]

  i = M

  while i < len(dic_result_sorted) and list(dic_result_sorted.values())[i] == list(dic_result_sorted.values())[i-1]:
      dic_final[list(dic_result_sorted.keys())[i]] = list(dic_result_sorted.values())[i]
      i += 1
  # print(f'dic_final: {dic_final}')
  return dic_final

def aminoacids_new(M,dic_result_sorted):
    AminoAcidMass = {}
    dic_intermediario = get_M(M,dic_result_sorted)
  
    for i in dic_intermediario.keys():
        AminoAcidMass[chr(i)] = int(i)
    return AminoAcidMass

def ConvolutionCyclopeptideSequencing(M, N, Spectrum):
    Resultado = format_ans(Spectrum)
    nova_lista = []
    
    for massa in Resultado:
        if massa >= 57 and massa <= 200:
            nova_lista.append(massa)
    
    result = Counter(nova_lista)
    dic_result = dict(result)
    # print(f'dic_result: {dic_result}')
    dic_result_sorted = dict(sorted(dic_result.items(), key=lambda item: item[1], reverse=True))
    # print(f'dic_result_sorted: {dic_result_sorted}')
    
    AminoAcidMass = aminoacids_new(M,dic_result_sorted)
    # print(f'AminoAcidMass: {AminoAcidMass}')
    Alphabet =  list(AminoAcidMass.keys())
    # print(f'Alphabet: {Alphabet}')
    
    result_peptides = LeaderboardCyclopeptideSequencing(Spectrum, N, Alphabet, AminoAcidMass)
    formatted_result = " ".join(format_peptide(peptide, AminoAcidMass) for peptide in result_peptides)
    return formatted_result, AminoAcidMass


with open("dataset_30246_8.txt", "r") as file:
    lines = file.readlines()
    M = int(lines[0].strip())
    N = int(lines[1].strip())
    Spectrum = list(map(int, lines[2].strip().split())) 

formatted_result, AminoAcidMass = ConvolutionCyclopeptideSequencing(M,N,Spectrum)
print(formatted_result)

57-128-163-99-128-113-147-97-186-147-57 57-129-147-114-128-163-99-128-113-147-97 58-128-147-114-128-163-99-128-113-147-97 58-147-114-128-163-99-128-113-147-97-128 73-113-147-114-128-163-99-128-113-147-97 97-73-113-147-114-128-163-99-128-113-147 97-147-113-128-99-163-128-57-57-147-57-129 97-147-113-128-99-163-128-57-57-147-58-128 97-147-113-128-99-163-128-57-57-147-73-113 97-147-113-128-99-163-128-57-57-147-113-73 97-147-113-128-99-163-128-57-57-147-128-58 97-147-113-128-99-163-128-57-57-147-129-57 97-147-113-128-99-163-128-57-57-147-186 97-147-113-128-99-163-128-114-147-57-129 97-147-113-128-99-163-128-114-147-58-128 97-147-113-128-99-163-128-114-147-73-113 97-147-113-128-99-163-128-114-147-113-73 97-147-113-128-99-163-128-114-147-128-58 97-147-113-128-99-163-128-114-147-129-57 97-147-113-128-99-163-128-114-147-186 97-186-147-57-57-128-163-99-128-113-147 97-186-147-114-128-163-99-128-113-147 99-128-113-147-97-186-147-114-128-163 99-163-128-57-57-147-186-97-147-113-128 99-163-128-114-14